In [3]:
import json

class Object:
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

In [4]:
diseases = ["none", "flu", "tetanus", "chickenpox", "whooping cough"]
probs = [0.3, 0.2, 0.1, 0.3, 0.1]

In [5]:
import random
import math
import time

class Moskito:
    def __init__(self, id, name, base, speed = 6, fuel = 160):
        self.id = id
        self.name = name
        self.base = base
        self.pos = base
        self.state = 1
        self.target = self.focus()
        self.lifetime = 0
        
        self.speed = speed
        self.maxSpeed = speed

        self.fuel = fuel
        self.maxFuel = fuel
        
        self.diseases = "empty"
        self.timestamp = time.time()
        
        self.temperature = 10
        self.weather = "rain"
        
    def update(self, temperatures, weathers):
        self.lifetime += (time.time() - self.timestamp)
        self.timestamp = time.time()
        self.speed = self.maxSpeed
        
        if self.fuel <= 0:
            return "fuel"
        
        if self.pos[0] > 0 and self.pos[1] > 0:
            self.temperature = temperatures[0]
            self.temperature -= 5
            self.weather = weathers[0]
        elif self.pos[0] > 0 and self.pos[1] < 0:
            self.temperature = temperatures[1]
            self.temperature -= 5
            self.weather = weathers[1]
        elif self.pos[0] < 0 and self.pos[1] < 0:
            self.temperature = temperatures[2]
            self.weather = weathers[2]
        else:
            self.temperature = temperatures[3]
            self.weather = weathers[3]
        
        if self.pos[1] > 40:
            self.temperature -= 10
            
        if self.temperature < 8:
            return "cold"
        
        if self.weather == "wind" and random.random() < 0.1:
            return "wind"
        
        if self.state == 0:
            # Base
            self.target = self.focus()
            self.state = 1
            self.fuel = self.maxFuel
            self.diseases = "empty"
        elif self.state == 1:
            # Go
            if self.temperature < 0:
                self.fuel -= self.speed * 4
            elif self.temperature < 10:
                self.fuel -= self.speed * 2
            elif self.temperature < 20:
                self.fuel -= self.speed * 1.5
            else:
                self.fuel -= self.speed
                
            if self.weather == "rain" or self.weather == "snow":
                self.speed /= 2
            
            vecDist = self.getVecDistance(self.target)
            self.pos = (self.pos[0] + vecDist[0] * self.speed + random.random() * 2 - 1, 
                        self.pos[1] + vecDist[1] * self.speed + random.random() * 2 - 1)
            
            if self.getDistance(self.target) <= 3:
                self.diseases = self.getDiseases()
                self.state = 2
        elif self.state == 2:
            # Return
            if self.temperature < 0:
                self.fuel -= self.speed * 8
            elif self.temperature < 10:
                self.fuel -= self.speed * 4
            elif self.temperature < 20:
                self.fuel -= self.speed * 2
            else:
                self.fuel -= self.speed
                
            if self.weather == "rain" or self.weather == "snow":
                self.speed /= 3
            
            vecDist = self.getVecDistance(self.base)
            self.pos = (self.pos[0] + vecDist[0] * self.speed, self.pos[1] + vecDist[1] * self.speed)
            
            if self.getDistance(self.base) <= 3:
                self.state = 0
        return self.information(False, "none")
    
    def information(self, die, cause):
        me = dict()
        me["id"] = self.id
        me["state"] = self.state
        me["name"] = self.name
        me["x"] = self.pos[0]
        me["y"] = self.pos[1]
        me["target_x"] = self.target[0]
        me["target_y"] = self.target[1]
        me["fuel"] = self.fuel
        me["maxfuel"] = self.maxFuel
        me["disease"] = self.diseases
        me["timestamp"] = time.time()
        me["lifetime"] = self.lifetime
        me["temperature"] = self.temperature
        me["weather"] = self.weather
        me["die"] = die
        me["cause"] = cause
        return me
    
    def focus(self):
        return (random.random() * 100 - 50, random.random() * 100 - 50)
    
    def getVecDistance(self, target):
        x = target[0] - self.pos[0]
        y = target[1] - self.pos[1]
        a = abs(x) + abs(y)
        if a == 0:
            return (0, 0)
        x /= a
        y /= a
        return (x, y)
    
    def getDistance(self, target):
        return math.sqrt((target[0] - self.pos[0])**2 + (target[1] - self.pos[1])**2)
    
    def getDiseases(self):
        rand = random.random()
        curr = 0
        disease = ""
        for i in range(0, len(diseases)):
            curr += probs[i]
            if rand < curr:
                return diseases[i]

In [7]:
import time

moskitos = []
capacity = 25
ids = 0
for i in range(0, capacity):
    moskitos.append(Moskito(i, "mos_" + str(ids), (0, 0)))
    ids += 1

moskitosJson = []
temperature = 20
weathers = ["rain", "sun", "snow", "cloudy", "wind"]
weatherLocs = []
temperaturesLocs = []
open('./data.json', 'w').close()
for i in range(0,10000):
    if i % 50 == 0:
        if random.random() > 0.5:
            temperature += 1
        else:
            temperature -= 1
        weatherLocs = []
        temperaturesLocs = []
        for j in range(0, 4):
            weather = int(random.random() * 5)
            weatherLocs.append(weathers[weather])
            if weathers[weather] == "rain":
                temperaturesLocs.append(temperature - 5)
            elif weathers[weather] == "snow":
                temperaturesLocs.append(temperature - 10)
            elif weathers[weather] == "sun":
                temperaturesLocs.append(temperature + 5)
            else:
                temperaturesLocs.append(temperature)

    if len(moskitos) < capacity:
        moskitos.append(Moskito(ids, "mos_" + str(ids), (0, 0)))
        ids += 1
    for moskito in moskitos:
        moskitoJson = moskito.update(temperaturesLocs, weatherLocs)
        if moskitoJson == "fuel":
            moskitosJson.append(moskito.information(True, "fuel"))
            moskitos.remove(moskito)
        elif moskitoJson == "wind":
            moskitosJson.append(moskito.information(True, "wind"))
            moskitos.remove(moskito)
        elif moskitoJson == "cold":
            moskitosJson.append(moskito.information(True, "cold"))
            moskitos.remove(moskito)
        else:
            moskitosJson.append(moskitoJson)
with open('./data.json', "a") as f:
    json.dump(moskitosJson, f)